<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- Creation of Ifrs Variables
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the creation of Ifrs Variable from the Scopes.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "4ImportScope-TechnicalMargin"

# Computing All Scopes

Since all the calculations defined in the various scopes are to be performed together, it is useful to introduce the scopes: <code>ComputeIfrsVarsActuals</code>, <code>ComputeIfrsVarsCashflows</code> and the <code>ComputeIfrsVarsOpenings</code> below, which given a certain Import Format and Import Storage, performs all the calculations (defined above) based on the corresponding data:

In [0]:
public interface PvToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> PvLocked => GetScope<PvLocked>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });

    IEnumerable<IfrsVariable> PvCurrent => GetScope<PvCurrent>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
}

In [0]:
public interface NominalToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> CumulatedNominal => GetScope<CumulatedNominalBE>(Identity).PresentValues
    .Concat(GetScope<CumulatedNominalRA>(Identity).PresentValues)
    .Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod), 
                        Partition = GetStorage().TargetPartition });
}

In [0]:
public interface RaToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> RaCurrent => GetScope<RaCurrent>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = null,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
                        
    IEnumerable<IfrsVariable> RaLocked => GetScope<RaLocked>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = null,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
}

In [0]:
public interface ActualToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> Actual => GetScope<Actual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
                        
    IEnumerable<IfrsVariable> AdvanceActual => GetScope<AdvanceActual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });

   IEnumerable<IfrsVariable> OverdueActual => GetScope<OverdueActual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
}

In [0]:
public interface DeferrableToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> DeferrableActual => GetScope<DeferrableActual>(Identity).RepeatOnce().Select(x => 
        new IfrsVariable{ EstimateType = x.EstimateType,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          AccidentYear = null,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition });
}

In [0]:
public interface EaForPremiumToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> BeEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                                                || Identity.IsReinsurance
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<BeExperienceAdjustmentForPremium>(Identity)
                              .ByAmountType
                              .Select(sc => new IfrsVariable{ EstimateType = GetScope<BeExperienceAdjustmentForPremium>(Identity).EstimateType, 
                                                              DataNode = sc.Identity.Id.DataNode, 
                                                              AocType = sc.Identity.Id.AocType, 
                                                              Novelty = sc.Identity.Id.Novelty, 
                                                              AccidentYear = sc.Identity.AccidentYear,
                                                              EconomicBasis = sc.EconomicBasis,
                                                              AmountType = sc.Identity.AmountType,
                                                              Values = SetProjectionValue(sc.Value, sc.Identity.Id.ProjectionPeriod),
                                                              Partition = sc.GetStorage().TargetPartition });
    
    IEnumerable<IfrsVariable> ActEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                                                 || Identity.IsReinsurance
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<ActualExperienceAdjustmentOnPremium>(Identity)
                                .ByAmountTypeAndEstimateType
                                .Select(sc => new IfrsVariable{ EstimateType = GetStorage().ExperienceAdjustEstimateTypeMapping[sc.Identity.EstimateType], 
                                                 DataNode = sc.Identity.Id.DataNode, 
                                                 AocType = sc.Identity.Id.AocType, 
                                                 Novelty = sc.Identity.Id.Novelty, 
                                                 AccidentYear = sc.Identity.AccidentYear,
                                                 //EconomicBasis = scope.EconomicBasis,
                                                 AmountType = sc.Identity.AmountType,
                                                 Values = SetProjectionValue(sc.Value, sc.Identity.Id.ProjectionPeriod),
                                                 Partition = GetStorage().TargetPartition });
}

In [0]:
public interface TmToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => Identity.ValuationApproach == ValuationApproaches.VFA ? EconomicBases.C : EconomicBases.L;
    IEnumerable<IfrsVariable> AmortizationFactor => Identity.AocType == AocTypes.AM
                                                                            ? GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis))
                                                                                .RepeatOnce()
                                                                                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                                                               DataNode = x.Identity.DataNode,
                                                                                                               AocType = x.Identity.AocType,
                                                                                                               Novelty = x.Identity.Novelty,
                                                                                                               EconomicBasis = x.EconomicBasis,
                                                                                                               Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                                                                                               Partition = GetStorage().TargetPartition
                                                                                                               })
                                                                            : Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> Csms => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<ContractualServiceMargin>(Identity).RepeatOnce()
                              .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                   DataNode = x.Identity.DataNode,
                                   AocType = x.Identity.AocType,
                                   Novelty = x.Identity.Novelty,
                                   Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                   Partition = GetStorage().TargetPartition
                                });

     IEnumerable<IfrsVariable> Loss => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                             ? Enumerable.Empty<IfrsVariable>()
                             : Identity.IsReinsurance 
                                ? GetScope<LossRecoveryComponent>(Identity).RepeatOnce()
                                    .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                   DataNode = x.Identity.DataNode,
                                                                   AocType = x.Identity.AocType,
                                                                   Novelty = x.Identity.Novelty,
                                                                   Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                                                   Partition = GetStorage().TargetPartition
                                                                })
                                : GetScope<LossComponent>(Identity).RepeatOnce()
                                    .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                   DataNode = x.Identity.DataNode,
                                                                   AocType = x.Identity.AocType,
                                                                   Novelty = x.Identity.Novelty,
                                                                   Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                                                   Partition = GetStorage().TargetPartition
                                                                });
}